In [ ]:
from hashlib import sha1
import hmac
import binascii
def getUrl(request):
    devId = 3002228
    key = '5bef2099-6493-48e4-b344-e37e99530ce6'
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))


import requests
import json
def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)


routes_json_file = '../data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    stops_files = [open(f'../data/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]
    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()  

In [188]:
def to_lower(text):
    text = text.lower()
    return text

In [529]:
import pandas as pd
train_stops_file = pd.read_json('../data/stops_file_0.json')
train_stops_file = train_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(to_lower)
train_stops_file['stop_name'] = train_stops_file['stop_name'].drop_duplicates()

In [530]:
train_stops_file = train_stops_file.dropna()
train_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ashburton,-37.868317,145.079666,1002,Alamein Station
1,ashburton,-37.861970,145.081345,1010,Ashburton Station
2,hawthorn east,-37.822400,145.045837,1012,Auburn Station
3,burnley,-37.827557,145.007553,1030,Burnley Station
4,glen iris,-37.851562,145.080500,1031,Burwood Station
...,...,...,...,...,...
347,williams landing,-37.869870,144.747452,1225,Williams Landing Station
348,yarraville,-37.815850,144.889938,1216,Yarraville Station
355,williamstown,-37.857330,144.889069,1146,North Williamstown Station
361,williamstown,-37.867756,144.905319,1211,Williamstown Station


In [197]:
def change_name(text):
    if(text == 'melbourne city'):
        return 'melbourne'
    else:
        return text
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(change_name)

In [364]:
tram_stops_file = pd.read_json('../data/stops_file_1.json')
tram_stops_file = tram_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(to_lower)
tram_stops_file['stop_name'] = tram_stops_file['stop_name'].drop_duplicates()
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(change_name)
tram_stops_file = tram_stops_file.dropna()
tram_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,brunswick east,-37.769787,144.971954,2227,Albert St/Lygon St #123
1,brunswick east,-37.761890,144.974060,2240,Albion St/Holmes St #127
2,brunswick east,-37.762370,144.973267,2231,Albion St/Lygon St #127
3,southbank,-37.821842,144.969513,2203,Arts Precinct/St Kilda Rd #14
4,southbank,-37.823814,144.967255,2266,Arts Precinct/Sturt St #17
...,...,...,...,...,...
1509,prahran,-37.850563,144.984039,2978,Punt Rd/High St #28
1516,coburg,-37.755560,144.964279,2236,Sydney Rd/Moreland Rd #132
1517,prahran,-37.852440,144.998100,2640,The Avenue/High St #33
1519,glen iris,-37.858078,145.041046,2622,Tooronga Rd/High St #48


In [365]:
bus_stops_file = pd.read_json('../data/stops_file_2.json')
bus_stops_file = bus_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(to_lower)
bus_stops_file['stop_name'] = bus_stops_file['stop_name'].drop_duplicates()
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(change_name)
bus_stops_file = bus_stops_file.dropna()
bus_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,avondale heights,-37.762234,144.859680,23570,Avondale Heights Reserve/Doyle St
1,maribyrnong,-37.769444,144.872772,31671,Canning Reserve/Canning St
3,avondale heights,-37.769413,144.857254,25271,Canning St/Herbert St
4,maribyrnong,-37.769013,144.877823,11239,Central Park Ave/Cordite Ave
5,avondale heights,-37.759260,144.857391,25277,Davis Ave/Skewes St
...,...,...,...,...,...
29797,hampton park,-38.030907,145.258270,19099,Robert Booth Reserve/Somerville Rd
29798,hampton park,-38.042637,145.270248,17513,Robert Ct/Huntington Dr
29799,narre warren south,-38.050587,145.296158,17528,Seebeck Dr/Sneddon Dr
29802,hampton park,-38.039837,145.280609,17583,The Ridge/Ralph Cres


In [366]:
vline_stops_file = pd.read_json('../data/stops_file_3.json')
vline_stops_file = vline_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(to_lower)
vline_stops_file['stop_name'] = vline_stops_file['stop_name'].drop_duplicates()
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(change_name)
vline_stops_file = vline_stops_file.dropna()
vline_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy
...,...,...,...,...,...
1381,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St
1383,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy
1391,clunes,-37.303142,143.781570,1591,Clunes Railway Station
1405,talbot,-37.172900,143.705627,1594,Talbot Railway Station


In [ ]:
from urllib import parse
import html2text as html2text
import requests
import json
url = "https://vahi.vic.gov.au/views/ajax?_wrapper_format=drupal_ajax"
for i in range(1):
    post_body = parse.urlencode({"view_name":"hospital_and_health_services","view_display_id":"page_1","page":i})
    response = requests.post(url= url,headers={
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "zh-CN,zh;q=0.9",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"104\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
      },data = post_body)
    print(response.status_code)
    print(response.text)
    with open(f'../data/v2_ajax_download_{i:02d}','w') as f:
        f.write(response.text)



with open('../data/v2_ajax_download_00') as f:
    d = json.load(f)
    print(len(d))
    geofield_google_map = d[0]["settings"]["geofield_google_map"]
    with open('../data/v2_ajax_download.csv','w') as csv_file:
        csv_file.write('"name","address"\n')
        for geofield in geofield_google_map:
            for feature in geofield_google_map[geofield]["data"]["features"]:
                #print(feature)
                name_with_tag = feature["properties"]["data"]["view_node"]
                name = name_with_tag[name_with_tag.index('>')+1 : name_with_tag.index('<',1)]
                address = feature["properties"]["data"]["field_address"]
                #print(html2text.html2text(),end=";")
                csv_file.write(f'"{name}","{address}"\n')

In [518]:
hospital = pd.read_csv("../data/v2_ajax_download.csv")


In [519]:
for i in range(len(hospital)):
    postcode_text = re.findall(r'VIC [0-9]+',hospital['address'][i])
    if len(postcode_text) == 1:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]
    elif len(postcode_text) == 0:
        hospital.loc[i,'postcode'] = None
    else:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]



In [521]:
hospital = hospital.dropna().reset_index()
hospital = hospital.drop('index',axis = 1)
hospital.to_csv('../data/hospital_csv')

In [355]:
geo_df = pd.read_excel("../data/postcode_in_VIC.csv")
geo_df = geo_df[geo_df['Postcode'] > 0 ]
geo_df = geo_df[geo_df['Postcode'] < 8000 ]
geo_df['Locality'] = geo_df['Locality'].apply(to_lower)


In [371]:
import math
EARTH_REDIUS = 6378.137


def rad(d):
    return d * math.pi / 180.0


def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2), 2) + math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s


def find_postcode(geo_df,lat, lng, suburb):
    if(suburb == 'melbourne'):
        distance = 10000
        result = None
        for index, item in geo_df.iterrows():
            postcode = item['Postcode']
            lat2, lng2 = item['Latitude'], item['Longitude']
            d = getDistance(lat, lng, lat2, lng2)
            if (d < distance):
                distance = d
                result = postcode
        return int(result)
    else:
        for index, item in geo_df.iterrows():
            locality = item['Locality']
            if locality == suburb:
                return int(item['Postcode'])


In [531]:
train_stops_file['postcode'] = None
for index, item in train_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    train_stops_file.loc[index, 'postcode'] = postcode


In [535]:
train_stops_file = train_stops_file.reset_index().drop('index', axis = 1)

In [372]:
tram_stops_file['postcode'] = None
for index, item in tram_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    tram_stops_file.loc[index, 'postcode'] = postcode
    

In [536]:
tram_stops_file = tram_stops_file.reset_index().drop('index', axis = 1)

In [373]:
bus_stops_file['postcode'] = None
for index, item in bus_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    bus_stops_file.loc[index, 'postcode'] = postcode

In [533]:
bus_stops_file = bus_stops_file.reset_index().drop('index', axis = 1)

In [374]:
vline_stops_file['postcode'] = None
for index, item in vline_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    vline_stops_file.loc[index, 'postcode'] = postcode

In [538]:
vline_stops_file = vline_stops_file.reset_index().drop('index', axis = 1)
vline_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name,postcode
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station,3377
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St,3377
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd,3286
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St,3300
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy,3294
...,...,...,...,...,...,...
634,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St,3714
635,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy,3660
636,clunes,-37.303142,143.781570,1591,Clunes Railway Station,3370
637,talbot,-37.172900,143.705627,1594,Talbot Railway Station,3371


In [539]:
train_stops_file.to_csv('../data/train_stops_file.csv')
tram_stops_file.to_csv('../data/tram_stops_file.csv')
bus_stops_file.to_csv('../data/bus_stops_file.csv')
vline_stops_file.to_csv('../data/vline_stops_file.csv')

In [126]:
import pandas as pd
import numpy as np
import mpu
import json
import re  
import geopandas as gpd
import folium
# import pygeos
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

# house = pd.read_csv('../data/house_combined.csv')
house = pd.read_csv('../data/house_combined.csv')
bus = pd.read_csv('../data/bus_stops_file.csv')
train = pd.read_csv('../data/train_stops_file.csv')
tram = pd.read_csv('../data/tram_stops_file.csv')
vline = pd.read_csv('../data/vline_stops_file.csv')
# population = pd.read_excel('../data/Population.xlsx')
# income = pd.read_excel('../data/Income.xlsx',sheet_name="Forecasted Income")
# male = pd.read_excel('../data/Sex & Age by Year.xlsx', sheet_name="Male")
# female = pd.read_excel('../data/Sex & Age by Year.xlsx', sheet_name="Female")
school = pd.read_csv('../data/school.csv',sep=",", encoding='cp1252')
hospital = pd.read_csv('../data/hospital_csv',sep=",", encoding='cp1252')
sc = pd.read_csv('../data/shopping_with_postcode.csv')
data_2019 = pd.read_csv('../data/2019_data.csv',sep=",", encoding='cp1252')
data_2020 = pd.read_csv('../data/2020_data.csv',sep=",", encoding='cp1252')
data_2021 = pd.read_csv('../data/2021_data.csv',sep=",", encoding='cp1252')
data_2022 = pd.read_csv('../data/2022_data.csv',sep=",", encoding='cp1252')
data_2022 = pd.read_csv('../data/2022_data.csv',sep=",", encoding='cp1252')
combine = pd.read_csv('../data/attribute_conbined.csv',sep=",", encoding='cp1252')
offence = pd.read_excel('../data/recorded offences.xlsx')
criminal = pd.read_excel('../data/criminal incidents2022-2013.xlsx')

In [127]:
need_col = [
    "Unnamed: 0",
    "propertyTypes",
    "bathrooms",
    "bedrooms",
    "carspaces",
    "dateUpdated",
    "dateListed",
    "geoLocation",
    "landAreaSqm",
    "postcode",
    "surburb",
    'year', 
    'month',
    'price', 
    'bond',
    'rent'
]

init_house = house[need_col]
init_house.rename(columns={"Unnamed: 0":"index"}, inplace=True)
init_house["geoLocation"] = init_house["geoLocation"].apply(lambda x: re.sub('\'','\"', str(x)))
init_house = init_house[init_house["geoLocation"] != 'nan']
init_house['longitude'] = init_house['geoLocation'].apply(lambda d: json.loads(d)["longitude"])
init_house['latitude'] = init_house['geoLocation'].apply(lambda d: json.loads(d)["latitude"])

In [128]:
df =  gpd.read_file("../data/SA2 ERP GeoPackage 2021 (ASGS2021).gpkg")

In [129]:
df = df[(df['State_name_2021'] == 'Victoria' )]
df = df[['SA2_maincode_2021','geometry']]

In [130]:
datashp = gpd.GeoDataFrame(init_house, geometry=gpd.points_from_xy(init_house.longitude,init_house.latitude))

In [131]:
init_house = gpd.sjoin(datashp, df, how='inner', op='intersects')

In [132]:
init_house

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021
0,0,['apartmentUnitFlat'],1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{""latitude"": -37.8341628, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,144.960873,-37.834163,POINT (144.96087 -37.83416),790,206051512
9,9,['house'],2.0,2.0,1.0,2019-01-11T17:10:21.69Z,2018-10-24T23:20:07Z,"{""latitude"": -37.8350754, ""longitude"": 144.964...",NaN,3205,south melbourne,2019,1,$725.00 per week,NaN,725.0,144.964862,-37.835075,POINT (144.96486 -37.83508),790,206051512
10,10,['apartmentUnitFlat'],1.0,2.0,0.0,2019-01-03T02:02:17.513Z,2018-10-25T00:57:25Z,"{""latitude"": -37.8402171, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$500.00 per week,NaN,500.0,144.960818,-37.840217,POINT (144.96082 -37.84022),790,206051512
11,11,['house'],1.0,3.0,0.0,2019-01-03T17:06:05.923Z,2018-10-25T22:30:15Z,"{""latitude"": -37.8366529, ""longitude"": 144.953...",NaN,3205,south melbourne,2019,1,$795.00 per week,NaN,795.0,144.953039,-37.836653,POINT (144.95304 -37.83665),790,206051512
21,21,['house'],2.0,2.0,0.0,2019-01-12T17:10:42.533Z,2018-10-31T22:21:33Z,"{""latitude"": -37.8375202, ""longitude"": 144.949...",NaN,3205,south melbourne,2019,1,$825.00 per week,NaN,825.0,144.949687,-37.837520,POINT (144.94969 -37.83752),790,206051512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114760,116020,['house'],1.0,3.0,1.0,2022-05-02T23:02:33.327Z,2022-04-06T22:38:28Z,"{""latitude"": -36.5365807, ""longitude"": 147.493...",NaN,3701,dartmouth,2022,5,$280,NaN,280.0,147.493795,-36.536581,POINT (147.49379 -36.53658),718,204031072
114766,116027,['house'],1.0,3.0,1.0,2022-06-28T07:34:34.59Z,2022-05-09T04:58:56Z,"{""latitude"": -36.5361943, ""longitude"": 147.494...",NaN,3701,dartmouth,2022,6,$290,NaN,290.0,147.494529,-36.536194,POINT (147.49453 -36.53619),718,204031072
114781,116052,['house'],1.0,3.0,2.0,2022-08-31T04:22:36.17Z,2022-08-18T06:04:32Z,"{""latitude"": -36.2218206, ""longitude"": 147.258...",NaN,3700,tallangatta,2022,8,$470,NaN,470.0,147.258393,-36.221821,POINT (147.25839 -36.22182),718,204031072
114974,116246,['house'],1.0,3.0,2.0,2021-03-09T00:53:53.08Z,2021-02-15T22:36:56Z,"{""latitude"": -36.2147158, ""longitude"": 147.177...",NaN,3700,tallangatta,2021,3,$310 per week,NaN,310.0,147.177222,-36.214716,POINT (147.17722 -36.21472),718,204031072


In [29]:
train = pd.read_csv('../data/train_stops_file.csv')
train = train[train['postcode'] > 0]

In [30]:
train_result = []
for index, values in train.iterrows():
    postcode = values['postcode']
    train_result.append([values['stop_suburb'],values['stop_latitude'],values['stop_longitude'],postcode])

In [31]:
train = pd.DataFrame(train_result, columns= ['stop_suburb','stop_latitude','stop_longitude','postcode'])
train['postcode'] = train["postcode"].apply(lambda x: int(x))
train

,stop_suburb,stop_latitude,stop_longitude,postcode
0,ashburton,-37.868317,145.079666,3147
1,ashburton,-37.861970,145.081345,3147
2,hawthorn east,-37.822400,145.045837,3123
3,burnley,-37.827557,145.007553,3121
4,glen iris,-37.851562,145.080500,3146
...,...,...,...,...
210,williams landing,-37.869870,144.747452,3027
211,yarraville,-37.815850,144.889938,3013
212,williamstown,-37.857330,144.889069,3016
213,williamstown,-37.867756,144.905319,3016


In [32]:
train_dict = dict()
for index, value in train.iterrows():
    postcode = value['postcode']
    value_list = train_dict.get(postcode,[])
    value_list.append(value)
    train_dict[postcode] = value_list

In [104]:
init_house = init_house.reset_index().drop('index', axis = 1)

In [105]:
init_house = init_house.reset_index().drop('index', axis = 1)
init_house = init_house.reset_index().drop('level_0', axis = 1)

In [ ]:
from math import radians, cos, sin, asin, sqrt
train_distance_list = []
train_count_list = []
for index, value in init_house.iterrows():
    print(index)
    postcode = value['postcode']
    longtitude = value['longitude']
    latitude = value['latitude']
    d = 100000000
    train_stops = train_dict.get(postcode)
    if train_stops is None:
        train_count_list.append(0)
    else:
        train_count_list.append(len(train_stops))
    for index2, values in train.iterrows():
        stop_latitude = values['stop_latitude']
        stop_longitude = values['stop_longitude']
        lng1, lat1, lng2, lat2 = map(radians, [float(longtitude), float(latitude), float(stop_longitude), float(stop_latitude)])
        dlon=lng2-lng1
        dlat=lat2-lat1
        a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        distance=2*asin(sqrt(a))*6371*1000 
        distance=round(distance/1000,3)
        if (distance < d):
            d = distance
    train_distance_list.append(d)

In [53]:
len(train_distance_list)

114960

In [106]:
init_house['train_distance'] = train_distance_list
init_house['train_count'] = train_count_list
init_house

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021,train_distance,train_count
0,0,['apartmentUnitFlat'],1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{""latitude"": -37.8341628, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,144.960873,-37.834163,POINT (144.96087 -37.83416),790,206051512,2.804,0
1,1,['house'],2.0,2.0,1.0,2019-01-11T17:10:21.69Z,2018-10-24T23:20:07Z,"{""latitude"": -37.8350754, ""longitude"": 144.964...",NaN,3205,south melbourne,2019,1,$725.00 per week,NaN,725.0,144.964862,-37.835075,POINT (144.96486 -37.83508),790,206051512,2.442,0
2,2,['apartmentUnitFlat'],1.0,2.0,0.0,2019-01-03T02:02:17.513Z,2018-10-25T00:57:25Z,"{""latitude"": -37.8402171, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$500.00 per week,NaN,500.0,144.960818,-37.840217,POINT (144.96082 -37.84022),790,206051512,2.751,0
3,3,['house'],1.0,3.0,0.0,2019-01-03T17:06:05.923Z,2018-10-25T22:30:15Z,"{""latitude"": -37.8366529, ""longitude"": 144.953...",NaN,3205,south melbourne,2019,1,$795.00 per week,NaN,795.0,144.953039,-37.836653,POINT (144.95304 -37.83665),790,206051512,3.457,0
4,4,['house'],2.0,2.0,0.0,2019-01-12T17:10:42.533Z,2018-10-31T22:21:33Z,"{""latitude"": -37.8375202, ""longitude"": 144.949...",NaN,3205,south melbourne,2019,1,$825.00 per week,NaN,825.0,144.949687,-37.837520,POINT (144.94969 -37.83752),790,206051512,3.544,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114955,114955,['house'],1.0,3.0,1.0,2022-05-02T23:02:33.327Z,2022-04-06T22:38:28Z,"{""latitude"": -36.5365807, ""longitude"": 147.493...",NaN,3701,dartmouth,2022,5,$280,NaN,280.0,147.493795,-36.536581,POINT (147.49379 -36.53658),718,204031072,233.799,0
114956,114956,['house'],1.0,3.0,1.0,2022-06-28T07:34:34.59Z,2022-05-09T04:58:56Z,"{""latitude"": -36.5361943, ""longitude"": 147.494...",NaN,3701,dartmouth,2022,6,$290,NaN,290.0,147.494529,-36.536194,POINT (147.49453 -36.53619),718,204031072,233.877,0
114957,114957,['house'],1.0,3.0,2.0,2022-08-31T04:22:36.17Z,2022-08-18T06:04:32Z,"{""latitude"": -36.2218206, ""longitude"": 147.258...",NaN,3700,tallangatta,2022,8,$470,NaN,470.0,147.258393,-36.221821,POINT (147.25839 -36.22182),718,204031072,240.828,0
114958,114958,['house'],1.0,3.0,2.0,2021-03-09T00:53:53.08Z,2021-02-15T22:36:56Z,"{""latitude"": -36.2147158, ""longitude"": 147.177...",NaN,3700,tallangatta,2021,3,$310 per week,NaN,310.0,147.177222,-36.214716,POINT (147.17722 -36.21472),718,204031072,236.377,0


In [62]:
hospital = pd.read_csv('../data/hospital_csv',sep=",", encoding='cp1252').drop('Unnamed: 0', axis = 1)
hospital_dict = dict()
for index, value in hospital.iterrows():
    postcode = value['postcode']
    value_list = hospital_dict.get(postcode,[])
    value_list.append(value)
    hospital_dict[postcode] = value_list

In [68]:
hospital_count_list = []
for index, value in init_house.iterrows():
    postcode = value['postcode']
    hospital_l = hospital_dict.get(postcode)
    if hospital_l is None:
        hospital_count_list.append(0)
    else:
        hospital_count_list.append(len(hospital_l))

In [107]:
init_house['hospital_count'] = hospital_count_list

In [77]:
sc = pd.read_csv('../data/shopping_with_postcode.csv')
a = sc['Location'].str.split(",",n = 1, expand = True)
sc['stop_longitude']=a[1]
b = a[0].str.split(":",n = 1, expand = True)
sc['stop_latitude']=b[1]

# update data type for lat and long extracted
sc = sc.astype({'stop_longitude': 'float'})
sc = sc.astype({'stop_latitude': 'float'}).drop(['Unnamed: 0','Name','Location'],axis = 1)
sc

,Suburb,postcode,stop_longitude,stop_latitude
0,Melbourne,3000,144.966947,-37.812733
1,St Kilda,3182,144.980617,-37.868967
2,Melbourne,3000,144.846270,-37.828989
3,Balnarring,3926,145.125010,-38.373199
4,Brunswick,3056,144.962004,-37.775695
...,...,...,...,...
184,Wodonga,3689,146.881917,-36.121153
185,Melton,3337,144.561270,-37.686167
186,Wodonga,3689,146.892456,-36.139008
187,Wyndham Vale,3024,144.607036,-37.888057


In [78]:
shopping_mall_dict = dict()
for index, value in sc.iterrows():
    postcode = value['postcode']
    value_list = shopping_mall_dict.get(postcode,[])
    value_list.append(value)
    shopping_mall_dict[postcode] = value_list

In [ ]:
merge_sc = pd.merge(init_house, sc, how='left',on='postcode')
from math import radians, cos, sin, asin, sqrt
shopping_mall_distance_list = []
shopping_mall_count_list = []
for index, value in init_house.iterrows():
    print(index)
    postcode = value['postcode']
    longtitude = value['longitude']
    latitude = value['latitude']
    d = 100000000
    shopping_mall = shopping_mall_dict.get(postcode)
    if shopping_mall is None:
        shopping_mall_count_list.append(0)
    else:
        shopping_mall_count_list.append(len(shopping_mall))
    for index2, values in sc.iterrows():
        stop_latitude = values['stop_latitude']
        stop_longitude = values['stop_longitude']
        lng1, lat1, lng2, lat2 = map(radians, [float(longtitude), float(latitude), float(stop_longitude), float(stop_latitude)])
        dlon=lng2-lng1
        dlat=lat2-lat1
        a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        distance=2*asin(sqrt(a))*6371*1000 
        distance=round(distance/1000,3)
        if (distance < d):
            d = distance
    shopping_mall_distance_list.append(d)

In [108]:
init_house['shopping_mall_distance'] = shopping_mall_distance_list
init_house['shopping_mall_count'] = shopping_mall_count_list
init_house

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021,train_distance,train_count,hospital_count,shopping_mall_distance,shopping_mall_count
0,0,['apartmentUnitFlat'],1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{""latitude"": -37.8341628, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,144.960873,-37.834163,POINT (144.96087 -37.83416),790,206051512,2.804,0,0,1.388,0
1,1,['house'],2.0,2.0,1.0,2019-01-11T17:10:21.69Z,2018-10-24T23:20:07Z,"{""latitude"": -37.8350754, ""longitude"": 144.964...",NaN,3205,south melbourne,2019,1,$725.00 per week,NaN,725.0,144.964862,-37.835075,POINT (144.96486 -37.83508),790,206051512,2.442,0,0,1.714,0
2,2,['apartmentUnitFlat'],1.0,2.0,0.0,2019-01-03T02:02:17.513Z,2018-10-25T00:57:25Z,"{""latitude"": -37.8402171, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$500.00 per week,NaN,500.0,144.960818,-37.840217,POINT (144.96082 -37.84022),790,206051512,2.751,0,0,1.927,0
3,3,['house'],1.0,3.0,0.0,2019-01-03T17:06:05.923Z,2018-10-25T22:30:15Z,"{""latitude"": -37.8366529, ""longitude"": 144.953...",NaN,3205,south melbourne,2019,1,$795.00 per week,NaN,795.0,144.953039,-37.836653,POINT (144.95304 -37.83665),790,206051512,3.457,0,0,1.304,0
4,4,['house'],2.0,2.0,0.0,2019-01-12T17:10:42.533Z,2018-10-31T22:21:33Z,"{""latitude"": -37.8375202, ""longitude"": 144.949...",NaN,3205,south melbourne,2019,1,$825.00 per week,NaN,825.0,144.949687,-37.837520,POINT (144.94969 -37.83752),790,206051512,3.544,0,0,1.370,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114955,114955,['house'],1.0,3.0,1.0,2022-05-02T23:02:33.327Z,2022-04-06T22:38:28Z,"{""latitude"": -36.5365807, ""longitude"": 147.493...",NaN,3701,dartmouth,2022,5,$280,NaN,280.0,147.493795,-36.536581,POINT (147.49379 -36.53658),718,204031072,233.799,0,0,68.639,0
114956,114956,['house'],1.0,3.0,1.0,2022-06-28T07:34:34.59Z,2022-05-09T04:58:56Z,"{""latitude"": -36.5361943, ""longitude"": 147.494...",NaN,3701,dartmouth,2022,6,$290,NaN,290.0,147.494529,-36.536194,POINT (147.49453 -36.53619),718,204031072,233.877,0,0,68.662,0
114957,114957,['house'],1.0,3.0,2.0,2022-08-31T04:22:36.17Z,2022-08-18T06:04:32Z,"{""latitude"": -36.2218206, ""longitude"": 147.258...",NaN,3700,tallangatta,2022,8,$470,NaN,470.0,147.258393,-36.221821,POINT (147.25839 -36.22182),718,204031072,240.828,0,1,32.738,0
114958,114958,['house'],1.0,3.0,2.0,2021-03-09T00:53:53.08Z,2021-02-15T22:36:56Z,"{""latitude"": -36.2147158, ""longitude"": 147.177...",NaN,3700,tallangatta,2021,3,$310 per week,NaN,310.0,147.177222,-36.214716,POINT (147.17722 -36.21472),718,204031072,236.377,0,1,25.565,0


In [113]:
combine = pd.read_csv('../data/attribute_conbined.csv',sep=",", encoding='cp1252')
combine.drop(['Unnamed: 0','surburb'], axis=1,inplace=True)
combine = combine.groupby('postcode').sum()
combine

,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university,garden,park,playground,pitch
postcode,,,,,,,,,,,,,,
0,0.0,0.0,1.0,2.0,3.0,21.0,0.0,2.0,4.0,0.0,0.0,25.0,7.0,31.0
3000,31.0,20.0,0.0,8.0,1.0,56.0,13.0,4.0,3.0,0.0,3.0,15.0,4.0,6.0
3002,102.0,19.0,0.0,1.0,1.0,55.0,8.0,10.0,1.0,0.0,30.0,23.0,4.0,8.0
3003,150.0,40.0,2.0,27.0,14.0,422.0,49.0,18.0,34.0,0.0,10.0,94.0,59.0,144.0
3004,86.0,50.0,0.0,10.0,1.0,128.0,86.0,6.0,2.0,8.0,46.0,28.0,6.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,74.0,48.0,2.0,62.0,12.0,552.0,64.0,29.0,45.0,10.0,19.0,172.0,65.0,308.0
3978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3980,29.0,18.0,3.0,18.0,2.0,118.0,14.0,7.0,13.0,0.0,4.0,26.0,18.0,61.0


In [118]:
init_house[init_house['rent'] == 0]

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021,train_distance,train_count,hospital_count,shopping_mall_distance,shopping_mall_count
21695,21695,['apartmentUnitFlat'],1.0,1.0,1.0,2021-10-05T05:42:00.45Z,2021-09-27T00:56:07Z,"{""latitude"": -37.8852577, ""longitude"": 145.002...",NaN,3185,elsternwick,2021,10,$350per week ($1521 pcm),NaN,0.0,145.002107,-37.885258,POINT (145.00211 -37.88526),841,208021179,0.120,2,1,2.615,0
48070,48070,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-06T07:08:14.49Z,2022-04-11T05:39:32Z,"{""latitude"": -37.8926173, ""longitude"": 145.065...",NaN,3163,murrumbeena,2022,6,"$380per week, $1651 per month",NaN,0.0,145.065575,-37.892617,POINT (145.06558 -37.89262),838,208021176,0.312,3,0,1.072,1
48828,48828,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-13T13:52:35.23Z,2021-12-08T02:27:38Z,"{""latitude"": -37.82578, ""longitude"": 145.047729}",NaN,3123,hawthorn east,2022,6,"$390per week, $1695 per month",NaN,0.0,145.047729,-37.825780,POINT (145.04773 -37.82578),811,207011152,0.411,1,1,0.987,0
80053,80053,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0
80144,80144,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0
100952,100952,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3
101015,101015,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3


In [123]:
house = pd.merge(init_house, combine, how='left',on='postcode')
house[house['rent'] == 0]

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021,train_distance,train_count,hospital_count,shopping_mall_distance,shopping_mall_count,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university,garden,park,playground,pitch
21695,21695,['apartmentUnitFlat'],1.0,1.0,1.0,2021-10-05T05:42:00.45Z,2021-09-27T00:56:07Z,"{""latitude"": -37.8852577, ""longitude"": 145.002...",NaN,3185,elsternwick,2021,10,$350per week ($1521 pcm),NaN,0.0,145.002107,-37.885258,POINT (145.00211 -37.88526),841,208021179,0.120,2,1,2.615,0,32.0,29.0,2.0,30.0,1.0,164.0,53.0,7.0,12.0,0.0,5.0,71.0,18.0,249.0
48070,48070,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-06T07:08:14.49Z,2022-04-11T05:39:32Z,"{""latitude"": -37.8926173, ""longitude"": 145.065...",NaN,3163,murrumbeena,2022,6,"$380per week, $1651 per month",NaN,0.0,145.065575,-37.892617,POINT (145.06558 -37.89262),838,208021176,0.312,3,0,1.072,1,31.0,19.0,0.0,11.0,9.0,122.0,23.0,5.0,12.0,0.0,3.0,31.0,17.0,51.0
48828,48828,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-13T13:52:35.23Z,2021-12-08T02:27:38Z,"{""latitude"": -37.82578, ""longitude"": 145.047729}",NaN,3123,hawthorn east,2022,6,"$390per week, $1695 per month",NaN,0.0,145.047729,-37.825780,POINT (145.04773 -37.82578),811,207011152,0.411,1,1,0.987,0,5.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,1.0,0.0,0.0,5.0,1.0,3.0
80053,80053,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0,49.0,4.0,4.0,12.0,13.0,111.0,10.0,3.0,12.0,0.0,12.0,86.0,39.0,72.0
80144,80144,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0,49.0,4.0,4.0,12.0,13.0,111.0,10.0,3.0,12.0,0.0,12.0,86.0,39.0,72.0
100952,100952,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3,114.0,13.0,2.0,16.0,9.0,310.0,18.0,11.0,24.0,1.0,77.0,119.0,48.0,190.0
101015,101015,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3,114.0,13.0,2.0,16.0,9.0,310.0,18.0,11.0,24.0,1.0,77.0,119.0,48.0,190.0


In [115]:
offence = offence.iloc[:,[1,3]].groupby('Postcode').sum()
criminal = criminal.iloc[:,[1,3]].groupby('Postcode').sum()
crim = pd.merge(offence, criminal, on='Postcode')
crim = crim.reset_index()
crim.columns=['postcode', 'offence count','incidents recorded']
crim

,postcode,offence count,incidents recorded
0,3000,161725,125561
1,3002,9970,7785
2,3003,9869,7846
3,3004,51905,25831
4,3006,35302,26069
...,...,...,...
688,3990,60,40
689,3991,496,357
690,3992,555,383
691,3995,11063,8118


In [124]:
house = pd.merge(house, crim, how='left', on='postcode')
house

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,geometry,index_right,SA2_maincode_2021,train_distance,train_count,hospital_count,shopping_mall_distance,shopping_mall_count,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university,garden,park,playground,pitch,offence count,incidents recorded
21695,21695,['apartmentUnitFlat'],1.0,1.0,1.0,2021-10-05T05:42:00.45Z,2021-09-27T00:56:07Z,"{""latitude"": -37.8852577, ""longitude"": 145.002...",NaN,3185,elsternwick,2021,10,$350per week ($1521 pcm),NaN,0.0,145.002107,-37.885258,POINT (145.00211 -37.88526),841,208021179,0.120,2,1,2.615,0,32.0,29.0,2.0,30.0,1.0,164.0,53.0,7.0,12.0,0.0,5.0,71.0,18.0,249.0,8126.0,6477.0
48070,48070,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-06T07:08:14.49Z,2022-04-11T05:39:32Z,"{""latitude"": -37.8926173, ""longitude"": 145.065...",NaN,3163,murrumbeena,2022,6,"$380per week, $1651 per month",NaN,0.0,145.065575,-37.892617,POINT (145.06558 -37.89262),838,208021176,0.312,3,0,1.072,1,31.0,19.0,0.0,11.0,9.0,122.0,23.0,5.0,12.0,0.0,3.0,31.0,17.0,51.0,15407.0,12022.0
48828,48828,['apartmentUnitFlat'],1.0,1.0,1.0,2022-06-13T13:52:35.23Z,2021-12-08T02:27:38Z,"{""latitude"": -37.82578, ""longitude"": 145.047729}",NaN,3123,hawthorn east,2022,6,"$390per week, $1695 per month",NaN,0.0,145.047729,-37.825780,POINT (145.04773 -37.82578),811,207011152,0.411,1,1,0.987,0,5.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,1.0,0.0,0.0,5.0,1.0,3.0,6735.0,5401.0
80053,80053,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0,49.0,4.0,4.0,12.0,13.0,111.0,10.0,3.0,12.0,0.0,12.0,86.0,39.0,72.0,27192.0,21030.0
80144,80144,['house'],1.0,3.0,2.0,2021-06-29T22:39:31.43Z,2021-02-23T22:36:54Z,"{""latitude"": -37.6980763, ""longitude"": 144.916...",NaN,3046,glenroy,2021,6,$395per week/ $1716,NaN,0.0,144.916248,-37.698076,POINT (144.91625 -37.69808),917,210031535,0.329,4,0,2.004,0,49.0,4.0,4.0,12.0,13.0,111.0,10.0,3.0,12.0,0.0,12.0,86.0,39.0,72.0,27192.0,21030.0
100952,100952,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3,114.0,13.0,2.0,16.0,9.0,310.0,18.0,11.0,24.0,1.0,77.0,119.0,48.0,190.0,55237.0,41689.0
101015,101015,['house'],2.0,5.0,2.0,2022-09-12T09:10:18.707Z,2022-08-23T03:41:04Z,"{""latitude"": -37.542691, ""longitude"": 144.9687...",448.0,3064,donnybrook,2022,9,$520per week/ $2260 pcm,NaN,0.0,144.968741,-37.542691,POINT (144.96874 -37.54269),889,209041225,6.957,2,1,5.602,3,114.0,13.0,2.0,16.0,9.0,310.0,18.0,11.0,24.0,1.0,77.0,119.0,48.0,190.0,55237.0,41689.0


In [125]:
house.to_csv('../data/new.csv')